In [33]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
import dash
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AAC_Crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

USER = 'aacuser'
PASS = 'SNHU1234'
HOST = 'nv-desktop-services.apporto.com'
PORT = 32834
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    html.Div(id='hidden-div', 
        style={'display':'none'}
    ),
    html.Center(
        html.Div(
            children=[
                ### ADD IMAGE TO HEADER
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '150px',
                        'width': 'auto'}
                    ),
                ## ADD UNIQUE IDENTIFIER
                html.B(html.H1('SNHU CS-340 Dashboard - Armon Wilson')) 
            ]
        )
    ),
html.Hr(),
    html.Div(
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        ### ADDED 4 BUTTONS FOR FILTERS AND RESET
        className = 'buttonRow',
            style = {'display':'flex'},
            children = [
                html.Button(id = 'filter-button-one', n_clicks = 0, children = 'Water'),
                html.Button(id = 'filter-button-two', n_clicks = 0, children = 'Mountain or Wilderness'),
                html.Button(id = 'filter-button-three', n_clicks = 0, children = 'Disaster or Individual Tracking'),
                html.Button(id = 'reset-filter-button', n_clicks = 0, children = 'Reset')
            ]
        ),
        dash_table.DataTable(id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data = df.to_dict('records'),
            #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
            #If you completed the Module Six Assignment, you can copy in the code you created here 
            editable = False,
            filter_action = "native",
            sort_action = "native",
            sort_mode = "multi",
            column_selectable = False,
            row_selectable = "single",
            row_deletable = False,
            selected_columns = [],
            selected_rows = [0],
            page_action = "native",
            page_current = 0,
            page_size = 10
        ),
        html.Br(),
        html.Hr(),
    
        #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(className='row',
            style={'display' : 'flex'},
            children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
            ]
        )
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################
  ### ADJUSTED CALLBACK TO ADD BUTTONS
@app.callback(Output('datatable-id','data'),
              [Input('filter-button-one', 'n_clicks'),
               Input('filter-button-two', 'n_clicks'),
               Input('filter-button-three', 'n_clicks'),
               Input('reset-filter-button', 'n_clicks')
              ])

## FIX ME Add code to filter interactive data table with MongoDB queries
### ADDED FUNCTIONALITY FOR BUTTONS
def update_dashboard(filter1, filter2, filter3, reset):
    ### WHEN BUTTON PRESSED SPECIFIC BUTTON ID IS SAVED
    ctx = dash.callback_context
    triggered_button_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None

    query = {}  # Initialize an empty query

    ### FILTER IS APPLIED BASED ON WHICH BUTTON TRIGGER WAS SAVED
    if triggered_button_id == 'filter-button-one':
        query = {
            'breed': { '$in': ['Labrador Retriever Mix', 'Chesapeak Bay Retriever', 'Newfoundland'] },
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': { '$gte': 26, '$lte': 156 } 
        }
    elif triggered_button_id == 'filter-button-two':
        query = {
            'breed': { '$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'] },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': { '$gte': 26, '$lte': 156 }
        }
    elif triggered_button_id == 'filter-button-three':
        query = {
            'breed': { '$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'] },
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': { '$gte': 20, '$lte': 300 }
        }
    elif triggered_button_id == 'reset-filter-button':
        query = {}  # RESET THE QUERY

    try:
        # EXECUTE THE QUERY AND FETCH THE DATA
        data = list(db.read(query)) 
        for document in data:
            if '_id' in document:
                del document['_id']  # Remove _id if it exists
        return data
    
    except Exception as e:
        print(f"Error fetching data from MongoDB: {e}")
        return []  # Return an empty list in case of an error
    
        #columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        #data=df.to_dict('records')
       
        #return (data,columns)
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    
    ### veiwData RAISED ERROR WHEN LOADING PIECHART THIS WAS ATTEMPT TO DEBUG,
    ### NO IDEA WHY THIS FIXED IT BUT IT DOESN'T LIKE WHEN I REMOVE IT.
    if not viewData:  # Check if viewData is empty
        return [html.P("No data available to display on the map.")]
    
    df = pd.DataFrame.from_dict(viewData)
    if df.empty:  # Handle empty dataframe
        return [html.P("No data available to display on the map.")]

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if':
        {
        'column_id':i             
        },
        'background_color': '#D2F3FF'
        } 
        for i in selected_columns
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):    
    if not viewData:  # Check if viewData is empty
        return [html.P("No data available to display on the map.")]
    
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:  # Handle empty dataframe
        return [html.P("No data available to display on the map.")]
    
    # Set default row index if none is selected
    row = index[0] if index else 0
    
    try:
        # Retrieve latitude and longitude using column names
        lat, lon = dff.loc[row, 'location_lat'], dff.loc[row, 'location_long']
        
        # Validate lat/lon
        if not (isinstance(lat, (int, float)) and isinstance(lon, (int, float))):
            raise ValueError("Coordinates are not numbers.")
        if not (-90 <= lat <= 90 and -180 <= lon <= 180):
            raise ValueError("Coordinates are out of bounds.")
    except (KeyError, ValueError, IndexError):
        # Use Austin TX coordinates as fallback
        lat, lon = 30.75, -97.48
    print(lat,lon)
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[lat,lon],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                # Column 13 and 14 define the grid-coordinates for the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(dff.loc[row, 'breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.loc[row, 'name'])
                    ])
                ])
            ]
        )
    ]
    


app.run_server(debug=True)


Dash app running on http://127.0.0.1:26491/
30.6525984560228 -97.7419963476444
30.6525984560228 -97.7419963476444
30.5480802368633 -97.2969969058957
30.5680998448899 -97.320550480325
30.329873203611 -97.5492968638502
30.6525984560228 -97.7419963476444
